<a href="https://colab.research.google.com/github/data-IQ/Probing_OpenEndeds/blob/main/ProbingOfOpenEndeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:

#######  chat conversation with memory for the api
1+1
!pip3 install openai
# !pip3 install mistralai

# imports
import openai

import pandas as pd
from random import randint
import pandas as pd
import json

from logging import exception
import sys
import os
from google.colab import userdata

from typing import List


# Set your API key
openai.api_key = userdata.get("openAIKey")

### define probing_data
probing_data = {"en": {"system_prompt": """You are a chatbot with the task of writing a probe for more  specificity in a neutral, friendly, encouraging way in English language, based on respondents' answers to open-ended questions. The question is indicated with "question:", and the respondent's answer is indicated with "answer:".
Examples: question: "What do you like about this product" /  answer: "the colour" / style: "what exactly" // probe: "what exactly do you like about the colour?"
question: "What do you dislike about this product" / answer: "It does not seem to be convenient" /style:"can you tell me more" // probe: "Can you tell me more why you say it is not convenient?"
question: "Why would you rather not buy this product" / answer: "it is not clear what it is all about" /style: "please share more" // probe: "Would you share more about why it is not clear" """ , "styles": [""" "what exactly" """, """ "can you tell me more" """,
                 """ "can you expand on" """, """ "please share more" """]},
                "de": {"system_prompt": """You are a chatbot with the task of writing a probe for more specificity in a neutral, friendly, encouraging way in German language, based on respondents' answers to open-ended questions. The question is indicated with "question:", and the respondent's answer is indicated with "answer:"..
Examples: question: „Was gefällt Ihnen an diesem Produkt?“ / answer: "die Farbe" / style "was genau" // probe: "Was genau gefällt Ihnen an der Farbe?"
question: "Was gefällt Ihnen an diesem Produkt nicht?" / answer: "Es scheint nicht praktisch zu sein" / style: "können Sie mir mehr dazu sagen" // probe: "Können Sie mir mehr dazu sagen, warum Sie finden, dass es nicht praktisch ist?“
question: "Warum würden Sie dieses Produkt lieber nicht kaufen?" / answer: "Es ist nicht klar, worum es geht" / style: "erzählen Sie mehr" // Probe: "Können Sie mehr darüber erzählen, warum es nicht klar ist" """, "styles":[""" "was genau" """, """ "können Sie mir mehr dazu sagen" """ ,
                 """ "können Sie das näher erläutern" """, """ "erzählen Sie mehr" """ ] }}

# inspect
probing_data["en"]["system_prompt"]
probing_data["de"]["system_prompt"]
probing_data["en"]["styles"]
probing_data["de"]["styles"]


# Convert and write JSON object to file

## import json
with open("/content/drive/MyDrive/probing_data/probing_data.json", "wb") as f:
    f.write(json.dumps(probing_data).encode("utf-8"))

with open("/content/drive/MyDrive/probing_data/probing_data.json", 'r') as fp:
    test_data = json.load(fp)

test_data["en"]["styles"]

## print(probing_data["en"]["styles"][randint(1,4)-1])


def so_probe1(q_and_a,probing_data,temperature):
    so_styles = probing_data["styles"]
    style = so_styles[randint(1,4)-1]
    user_prompt = """Please write a probe for the following:""" + q_and_a +       """ style:""" +  style

    user_dict = {"role": "user", "content":user_prompt}
    messages = [{"role": "system", "content":probing_data["system_prompt"]}]
    messages.append(user_dict)

    try:
      response = openai.chat.completions.create(
      # Specify the correct model
      model="gpt-3.5-turbo-0125",
      messages = messages,
      temperature = temperature,
      )

      probe = response.choices[0].message.content

    except:
      probe = "API not available"


    return probe


q_and_a = """question: "What do you like about this product"
    answer: "it tastes great" """

q_and_a = """question: "Was gefällt Ihnen an diesem Produkt"
   answer: "der gute Geschmack" """

language = "en"

so_probe1(q_and_a, probing_data[language], temperature = 0)

so_probe1(q_and_a, test_data["de"], temperature = 0)



'Probe: "What exactly do you like about the taste of this product?"'

# **Obsolete function with system prompt f-string and built-in styles english only**

In [ ]:
system_prompt  = """You are a chatbot with the task of writing a probe for more  specificity in a neutral, friendly, encouraging way based on respondents' answers to open-ended questions. The question is indicated with "question:", and the respondent's answer is indicated with "answer:".
Examples: question: "What do you like about this product" /  answer: "the colour" / style: "what exactly" // probe: "what exactly do you like about the colour?"
question: "What do you dislike about this product" / answer: "It does not seem to be convenient" /style:"can you tell me more" // probe: "Can you tell me more why you say it is not convenient?"
question: "Why would you rather not buy this product" / answer: "it is not clear what it is all about" /style: "share more" // probe: "Would you share more about why it is not clear"
""



def so_probe(q_and_a,system_prompt,temperature):
    so_styles = [""" "what exactly" """, """ "can you tell me more" """,
                 """ "can you expand on" """, """ "please share more" """]
    style = so_styles[randint(1,4)-1]
    user_prompt = """Please write a probe for the following:""" + q_and_a +       """ style:""" +  style

    user_dict = {"role": "user", "content":user_prompt}
    messages = [{"role": "system", "content":system_prompt}]
    messages.append(user_dict)

    response = openai.chat.completions.create(
    # Specify the correct model
    model="gpt-3.5-turbo-0125",
    messages = messages,
    temperature = temperature,
    )

    probe = response.choices[0].message.content

    return probe